<h1><center>NFL Big Data Bowl 2022 Insights of Special Teams Play</center></h1>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import datetime
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10,6)
import seaborn as sns
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### NFL Big Data Bowl 2022 data (.csv files) reading

In [ ]:
Players_df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/players.csv')
display(Players_df.shape)

In [ ]:
seasons = ["2018", "2019", "2020"]
tracking_df = pd.DataFrame()
for S in seasons:
    
    trackingTemp_df = pd.read_csv("../input/nfl-big-data-bowl-2022/tracking"+S+".csv")
    tracking_df = tracking_df.append(trackingTemp_df)

### NFL Big Data Bowl 2022 data Feature Engineering

In [ ]:
display(tracking_df.shape)
# removing outliers from player tracking data (keep data of Player position along the long axis x (sidelines)of the field, 0 - 120 yards and Player position along the short axis y (endzones lines)of the field, 0 - 53.3 yards)
tracking_df = tracking_df[(tracking_df.x>0) & (tracking_df.y>0)]
display(tracking_df.shape)
tracking_df.describe()

In [ ]:
plt.hist(tracking_df.x,bins=20,rwidth=0.8)
plt.xlabel('Player position on long axis(Yards)')
plt.ylabel('Count')
plt.show()
plt.hist(tracking_df.y,bins=20,rwidth=0.8)
plt.xlabel('Player position on short axis(Yards)')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.hist(tracking_df.x,bins=20,rwidth=0.8,density=True)
plt.xlabel('Player position on long axis(Yards)')
plt.ylabel('Count')

rng = np.arange(tracking_df.x.min(),tracking_df.x.max(),0.1)
plt.plot(rng,norm.pdf(rng,tracking_df.x.mean(),tracking_df.x.std()))

In [ ]:
plt.hist(tracking_df.y,bins=20,rwidth=0.8,density=True)
plt.xlabel('Player position on short axis(Yards)')
plt.ylabel('Count')

rng = np.arange(tracking_df.y.min(),tracking_df.y.max(),0.1)
plt.plot(rng,norm.pdf(rng,tracking_df.y.mean(),tracking_df.y.std()))

In [ ]:
print(f"\033[1m tracking_df x mean:  {tracking_df.x.mean()}")
print(f"\033[1m tracking_df y mean:  {tracking_df.y.mean()}")
print(f"\033[1m tracking_df x std:  {tracking_df.x.std()}")
print(f"\033[1m tracking_df y std:  {tracking_df.y.std()}")
x_upper_limit = tracking_df.x.mean() + 2*tracking_df.x.std()
print(f"\033[1m tracking_df x_upper_limit:  {x_upper_limit}")
x_lower_limit = tracking_df.x.mean() - 2*tracking_df.x.std()
print(f"\033[1m tracking_df x_lower_limit:  {x_lower_limit}")
y_upper_limit = tracking_df.y.mean() + 3*tracking_df.y.std()
print(f"\033[1m tracking_df y_upper_limit:  {y_upper_limit}")
y_lower_limit = tracking_df.y.mean() - 3*tracking_df.y.std()
print(f"\033[1m tracking_df y_lower_limit:  {y_lower_limit}")
print(f"\n \033[1m tracking_df shape before outliers remove:  {tracking_df.shape}\n")
#print(f"\n\033[1m tracking_df1 Outliers: \n\n")
#display(tracking_df1[(tracking_df1.x>x_lower_limit) | (tracking_df1.x<x_lower_limit) | (tracking_df1.y>y_upper_limit) | (tracking_df1.y<y_lower_limit)])
tracking_df1 = tracking_df[(tracking_df.x<x_upper_limit) & (tracking_df.x>x_lower_limit) & (tracking_df.y<y_upper_limit) & (tracking_df.y>y_lower_limit)]
print(f"\n \033[1m tracking_df1 shape after outliers remove:  {tracking_df1.shape}\n")
del tracking_df
tracking_df1.describe()

In [ ]:
tracking_df1_Speeds = tracking_df1.copy()
tracking_df1_Speeds = tracking_df1_Speeds[['nflId','gameId','playId','frameId','event','s']]
# Remove other events except 'kickoff','free_kick' from player tracking data
tracking_df1_Speeds = tracking_df1_Speeds.loc[tracking_df1_Speeds.groupby(['gameId','playId']).event.transform(lambda k: np.cumsum(k.isin(['kickoff','free_kick'])) >= 1)]
# Consider 45 frame ids of player speed from the moment of 'kickoff','free_kick' events 
tracking_df1_Speeds = tracking_df1_Speeds.groupby(['nflId','gameId','playId']).head(45).reset_index()
tracking_df1_Speeds = tracking_df1_Speeds.groupby(['nflId','gameId','playId']).s.apply(lambda z: z.max()).reset_index()
tracking_df1_Speeds = tracking_df1_Speeds.sort_values(by='s', ascending=False)
tracking_df1_Speeds.head(10)

In [ ]:
# Coverting Player's height from foot&inches to inches
#Players_df["height"] = Players_df["height"].str.replace("Jun", "6")
#Players_df["height"] = Players_df["height"].str.replace("May", "5")
ht = (Players_df.height.str.contains('-'), 'height')
Players_df.loc[ht]=Players_df.loc[ht].str.split('-').str[0].astype(int)*12+Players_df.loc[ht].str.split('-').str[1].astype(int)
Players_df['height'] = Players_df.height.astype(int)
#Players_df = Players_df.replace({'height':{6:72}})
Players_df["birthDate"] = pd.to_datetime(Players_df["birthDate"])
# finding Player age from birth date
Players_df["age"] = Players_df["birthDate"].apply(lambda l : (datetime.datetime.now().year - l.year))
display(Players_df.shape)
display(Players_df.describe())
Players_df.head(3)

In [ ]:
plt.hist(Players_df.height,bins=20,rwidth=0.8)
plt.xlabel('Height(inches)')
plt.ylabel('Count')
plt.show()
plt.hist(Players_df.age,bins=20,rwidth=0.8)
plt.xlabel('Age(years)')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.hist(Players_df.height,bins=20,rwidth=0.8,density=True)
plt.xlabel('Height(inches)')
plt.ylabel('Count')

rng = np.arange(Players_df.height.min(),Players_df.height.max(),0.1)
plt.plot(rng,norm.pdf(rng,Players_df.height.mean(),Players_df.height.std()))

In [ ]:
plt.hist(Players_df.age,bins=20,rwidth=0.8,density=True)
plt.xlabel('Age(Years)')
plt.ylabel('Count')

rng = np.arange(Players_df.age.min(),Players_df.age.max(),0.1)
plt.plot(rng,norm.pdf(rng,Players_df.age.mean(),Players_df.age.std()))

In [ ]:
print(f"\033[1m Players_df height mean:  {Players_df.height.mean()}")
print(f"\033[1m Players_df age mean:  {Players_df.age.mean()}")
print(f"\033[1m Players_df height std:  {Players_df.height.std()}")
print(f"\033[1m Players_df age std:  {Players_df.age.std()}")
height_upper_limit = Players_df.height.mean() + 2*Players_df.height.std()
print(f"\033[1m Players_df height_upper_limit:  {height_upper_limit}")
height_lower_limit = Players_df.height.mean() - 2*Players_df.height.std()
print(f"\033[1m Players_df height_lower_limit:  {height_lower_limit}")
age_upper_limit = Players_df.age.mean() + 3*Players_df.age.std()
print(f"\033[1m Players_df age_upper_limit:  {age_upper_limit}")
age_lower_limit = Players_df.age.mean() - 3*Players_df.age.std()
print(f"\033[1m Players_df age_lower_limit:  {age_lower_limit}")
print(f"\n \033[1m Players_df shape before outliers remove:  {Players_df.shape}\n")
#print(f"\n\033[1m Players_df Outliers: \n\n")
#display(Players_df[(Players_df.height>height_lower_limit) | (Players_df.height<height_lower_limit) | (Players_df.age>age_upper_limit) | (Players_df.age<age_lower_limit)])
Players_df1 = Players_df[(Players_df.height<height_upper_limit) & (Players_df.height>height_lower_limit) & (Players_df.age<age_upper_limit) & (Players_df.age>age_lower_limit)]
print(f"\n \033[1m Players_df shape after outliers remove:  {Players_df1.shape}\n")
display(Players_df1.describe())
Players_df1.head()

In [ ]:
# Merging Players & players tracking data after outliers removal
tracking_df1_Player_Speeds = pd.merge(tracking_df1_Speeds, Players_df1, left_on = ['nflId'], right_on = ['nflId'] )
tracking_df1_Player_Speeds = tracking_df1_Player_Speeds[['nflId','displayName','age','height','weight','s','gameId','playId']]
tracking_df1_Player_Speeds.head(5)

In [ ]:
plt.xlabel('Player Age')
plt.ylabel('Player Speed (Yards/Second)')
plt.scatter(tracking_df1_Player_Speeds['age'],tracking_df1_Player_Speeds['s'],color='green',marker='*')

In [ ]:
plt.xlabel('Player Height')
plt.ylabel('Player Speed (Yards/Second)')
plt.scatter(tracking_df1_Player_Speeds['height'],tracking_df1_Player_Speeds['s'],color='red',marker='*')

In [ ]:
plt.xlabel('Player Weight')
plt.ylabel('Player Speed (Yards/Second)')
plt.scatter(tracking_df1_Player_Speeds['weight'],tracking_df1_Player_Speeds['s'],color='Blue',marker='*')

### Model development

In [ ]:
tracking_df1_Player_Speeds_inputs = tracking_df1_Player_Speeds.drop(['nflId','displayName','gameId','playId','s'],axis='columns')
tracking_df1_Player_Speeds_inputs['age'] = tracking_df1_Player_Speeds_inputs.age.astype(int)
tracking_df1_Player_Speeds_inputs.head(3)

In [ ]:
tracking_df1_Player_Speeds_target = tracking_df1_Player_Speeds['s']
tracking_df1_Player_Speeds_target.head(3)

#### Use train_test_split to split data to train & test the model

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(tracking_df1_Player_Speeds_inputs,tracking_df1_Player_Speeds_target,test_size=0.3)
display(X_train.shape)
display(y_train.shape)
display(X_test.shape)
display(y_test.shape)

#### Use DecisionTreeRegressor algorithm to build the model

In [ ]:
tr_pl_dtr = DecisionTreeRegressor(random_state=0)
tr_pl_dtr.fit(X_train,y_train)
tr_pl_dtr.score(X_test,y_test)

#### Predict player speeds in yards/second with player age in years, height in inches & weight in lbs

In [ ]:
print(f"\033[1m Player with age 23 years, 74 inches height & 225 lbs weight predictable speed in yards per second after Kick Off or Free Kick:  {tr_pl_dtr.predict([[23, 74, 225]])}")
print(f"\033[1m Player with age 23 years, 75 inches height & 225 lbs weight predictable speed in yards per second after Kick Off or Free Kick:  {tr_pl_dtr.predict([[23, 75, 225]])}")
print(f"\033[1m Player with age 23 years, 74 inches height & 240 lbs weight predictable speed in yards per second after Kick Off or Free Kick:  {tr_pl_dtr.predict([[23, 74, 240]])}")
print(f"\033[1m Player with age 23 years, 75 inches height & 240 lbs weight predictable speed in yards per second after Kick Off or Free Kick:  {tr_pl_dtr.predict([[23, 75, 240]])}")
print(f"\033[1m Player with age 24 years, 75 inches height & 240 lbs weight predictable speed in yards per second after Kick Off or Free Kick:  {tr_pl_dtr.predict([[24, 75, 240]])}")
print(f"\033[1m Player with age 25 years, 75 inches height & 240 lbs weight predictable speed in yards per second after Kick Off or Free Kick:  {tr_pl_dtr.predict([[25, 75, 240]])}")
print(f"\033[1m Player with age 26 years, 75 inches height & 240 lbs weight predictable speed in yards per second after Kick Off or Free Kick:  {tr_pl_dtr.predict([[26, 75, 240]])}")
print(f"\033[1m Player with age 25.5 years, 75 inches height & 240 lbs weight predictable speed in yards per second after Kick Off or Free Kick:  {tr_pl_dtr.predict([[25.5, 75, 240]])}")
print(f"\033[1m Player with age 26 years, 74 inches height & 235 lbs weight predictable speed in yards per second after Kick Off or Free Kick:  {tr_pl_dtr.predict([[26, 74, 235]])}")
print(f"\033[1m Player with age 23.5 years, 72.5 inches height & 220 lbs weight predictable speed in yards per second after Kick Off or Free Kick:  {tr_pl_dtr.predict([[23.5, 72.5, 220]])}")
print(f"\033[1m Player with age 25 years, 77 inches height & 245 lbs weight predictable speed in yards per second after Kick Off or Free Kick:  {tr_pl_dtr.predict([[25, 77, 245]])}")
print(f"\033[1m Player with age 24 years, 74 inches height & 230 lbs weight predictable speed in yards per second after Kick Off or Free Kick:  {tr_pl_dtr.predict([[24, 73, 230]])}")

#### Deep Learning Model with two hidden layers

In [ ]:
tr_pl_kss = keras.Sequential([keras.layers.Dense(100,activation='softmax'),keras.layers.Dense(50,activation='relu'),keras.layers.Dense(25,activation='sigmoid')])
tr_pl_kss.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
tr_pl_kss.fit(X_train,y_train,epochs=100)

In [ ]:
tr_pl_kss.summary()

In [ ]:
tr_pl_kss.evaluate(X_test,y_test)

In [ ]:
y_predicted = tr_pl_kss.predict(X_test)
y_predicted_labels = [np.argmax(i) for i in y_predicted]
cm = tf.math.confusion_matrix(labels=y_test,predictions=y_predicted_labels)

plt.figure(figsize = (10,7))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')